In [ ]:
!pip install pytorch-lightning transformers 

In [1]:
import pandas as pd
import numpy as np
import os
import unicodedata
import torch

In [ ]:
from transformers import TFBertModel, BertModel, BertConfig,TFBertForSequenceClassification,BertForSequenceClassification, RobertaModel, RobertaForSequenceClassification
from transformers import BertTokenizer,RobertaTokenizer

from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import (
    random_split,
    DataLoader,
    RandomSampler,
    SequentialSampler,
    Subset,
    TensorDataset
)

from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tqdm.notebook import tqdm
import os
import json
import random

In [2]:
YelpZip_file_path = '/content/drive/MyDrive/YelpZip'

In [14]:
metadata = pd.read_csv(os.path.join(YelpZip_file_path, 'metadata'), sep= '\t',
                      names=['user_id', 'prod_id','rating','label', 'date'], header=None)
with open(os.path.join(YelpZip_file_path, 'reviewContent')) as r:
  content = r.readlines()

In [12]:
len(metadata)

608598

In [16]:
len(content)

608598

In [18]:
dataset = []
label = []
# id = {}
for text, lab in zip(content, metadata.itertuples(index=True, name='Pandas')):
  sentence = text.strip('\n').split('\t')[-1]
  sentence = unicodedata.normalize("NFKD", sentence)
  dataset.append(sentence)
  # id[text.user_id] = 1
  if lab.label == -1:
    l = 0
  else: 
    l = 1
  label.append(l)

In [19]:
len(dataset) 

608598

In [21]:
len(label)

608598

In [ ]:
true

527997

In [ ]:
def load_data():
    
    
    x_train, x_test, y_train, y_test = train_test_split(dataset,label, random_state=42,train_size=0.8) #train 80% test 20%

    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = load_data()

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    x_train,
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True,
    # is_split_into_words = True, 
    max_length=128, 
    return_tensors='pt'
)


encoded_data_test = tokenizer.batch_encode_plus(
    x_test, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=128, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(y_train)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(y_test)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
dataset_train

In [ ]:


model = RobertaForSequenceClassification.from_pretrained("roberta-base",
                                                      # problem_type="multi_label_classification",
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)




Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [ ]:

batch_size = 32
epochs = 3
learning_rate = 5e-5


In [ ]:

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

In [ ]:

optimizer = AdamW(model.parameters(),
                  lr=learning_rate, 
                  eps=1e-8)
                  


scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
def get_accuracy(pred, labels):
  preds_flat = np.argmax(pred, axis=1).flatten()
  labels_flat = labels.flatten()
  
  return accuracy_score(labels_flat, preds_flat)

# def get_binary_accuracy(pred, labels):
#   preds_flat = torch.round(torch.sigmiod(pred).flatten())
#   labels_flat = labels.flatten()
  
#   return accuracy_score(labels_flat, preds_flat)

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


device = torch.device("cuda:0")

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    


In [ ]:

CUDA_LAUNCH_BLOCKING=1

model.to(device)

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
     
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    # torch.save(model.state_dict(), f'/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_test)
    accuracy = get_accuracy(predictions, true_vals)
    tqdm.write(f'Testing loss: {val_loss}')
    tqdm.write(f'Accuracy: {accuracy}')

torch.save(model.state_dict(), f'/content/drive/MyDrive/RoBERTa_base_573.model')